# Library

In [1]:
import pandas as pd
import numpy as np
import os
import re
import collections
import nltk
from nltk.corpus import stopwords
import itertools 
from nltk.tokenize import word_tokenize
from string import punctuation

from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import unidecode

from bs4 import BeautifulSoup, Tag
import requests
import re
import urllib.parse
from urllib.parse import urlparse

# Functions

In [7]:
def read_texts(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.txt'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.txt',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

In [81]:
def read_texts2(path):
    data = []
    file_name = os.listdir(path)

    for name in file_name:
        if name.endswith('.tsv'):
            with open(path + name,encoding="utf8") as f:
                text = f.read()
                data.append({'nombre':name.replace('.tsv',''), 'texto':text})

    df = pd.DataFrame(data)
    return df

# Load Data

## BARR2

### Testing

220 clinical cases.

In [8]:
testing_abbr = pd.read_csv("../datasets/testing_set/clinical_cases.abbreviations.testing_set.tsv", sep = '\t')
testing_met = pd.read_csv("../datasets/testing_set/clinical_cases.metadata.testing_set.tsv", sep = '\t')
testing_rel = pd.read_csv("../datasets/testing_set/clinical_cases.relations.testing_set.tsv", sep = '\t')

In [9]:
testing_met.head()

,# Document_ID,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-01082006000700014-1.txt,1.txt,1130-0108,2006-07-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082007000300006-7.txt,7.txt,1130-0108,2007-03-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1134-80462009000100005-1.txt,1.txt,1134-8046,2009-02-01,Revista de la Sociedad Española del Dolor v.1...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1137-66272014000300015-1.txt,1.txt,1137-6627,2014-12-01,Anales del Sistema Sanitario de Navarra v.37 ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0365-66912004001200011-1.txt,1.txt,0365-6691,2004-12-01,Archivos de la Sociedad Española de Oftalmolog...,http://scielo.isciii.es/scielo.php?script=sci_...


In [10]:
testing_rel = testing_rel.reset_index()

In [11]:
testing_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [12]:
testing_rel = testing_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [13]:
testing_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0211-69952013000500019-1,SHORT_FORM,3739,ARA II,SHORT-LONG,LONG_FORM,3695,3737,antagonista del receptor de angiotesina II
1,S0211-69952013000500019-1,SHORT_FORM,2793,PCR,SHORT-LONG,LONG_FORM,2798,2837,reacción en cadena de enzima polimerasa
2,S0365-66912004000600008-1,SHORT_FORM,406,AV,SHORT-LONG,LONG_FORM,390,404,agudeza visual
3,S0211-69952012000500025-1,SHORT_FORM,945,angio-TAC,SHORT-LONG,LONG_FORM,908,943,angiotomografía computarizada axial
4,S1130-05582017000200122-1,SHORT_FORM,940,RMN,SHORT-LONG,LONG_FORM,910,938,resonancia magnética nuclear


In [14]:
testing_rel.Relation_type.unique()

array(['SHORT-LONG', 'SHORT-NESTED', 'NESTED-LONG'], dtype=object)

In [15]:
testing_abbr = testing_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [16]:
testing_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142010000500014-1,2037,2044,16SrRNA,16s ribosomal rna,16s ribosomal rno
1,S0004-06142010000500014-1,1349,1351,M.,mycobacterium,mycobacterium
2,S0004-06142010000500014-1,1339,1342,PCR,polymerase chain reaction,polymerase chain reaction
3,S0004-06142010000500014-1,611,615,BHCG,beta-human chorionic gonadotropin,beta-humar chorionic gonadotropin
4,S0004-06142010000500014-1,594,597,CEA,carcinoembrionary antigen,carcinoembrionary antiger


In [20]:
testing_raw = read_texts("../datasets/testing_set/testing_set.raw_text/txt/")

In [21]:
testing_raw.head()

,nombre,texto
0,S0004-06142005000900014-1,Varón de diecisiete años sin antecedentes de i...
1,S0004-06142006000500002-4,Paciente de 90 años que acude a su Urólogo de ...
2,S0004-06142006000500012-1,Se trata de un varón de 53 años sin antecedent...
3,S0004-06142006000600012-1,"Paciente de 62 años de edad,consulta por prese..."
4,S0004-06142007000600011-1,Paciente de 27 años de edad sin alergias ni an...


### Backgound

2879 clinical cases. 220 will be use for clinical evaluation

In [19]:
# back_met = pd.read_csv("../datasets/background_test/clinical_cases.metadata.background_set.tsv", sep = '\t')

In [20]:
# back_met = back_met.rename(columns = {'# Document_ID': 'doc_id'})

In [2]:
# back_met.head()

In [3]:
# back_raw = read_texts("../datasets/background_test/background_test_set/")

In [4]:
# back_raw.head()

### Development

146 clinical cases

In [22]:
dev_abbr = pd.read_csv("../datasets/development_set/clinical_cases.abbreviations.development_set.tsv", sep = '\t')
dev_met = pd.read_csv("../datasets/development_set/clinical_cases.metadata.development_set.tsv", sep = '\t')
dev_rel = pd.read_csv("../datasets/development_set/clinical_cases.relations.development_set.tsv", sep = '\t')

In [23]:
dev_met = dev_met.rename(columns = {'# Document_ID': 'doc_id'})

In [24]:
dev_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1130-14732005000300004-1.txt,1.txt,1130-1473,2005-06-01,Neurocirugía v.16 n.3 2005,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-01082008000200009-1.txt,1.txt,1130-0108,2008-02-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S1137-66272012000300021-1.txt,1.txt,1137-6627,2012-12-01,Anales del Sistema Sanitario de Navarra v.35 ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S1699-695X2016000200010-1.txt,1.txt,1699-695X,2016-06-01,Revista Clínica de Medicina de Familia v.9 n....,http://scielo.isciii.es/scielo.php?script=sci_...
4,S1130-01082007001100009-1.txt,1.txt,1130-0108,2007-11-01,Revista Española de Enfermedades Digestivas v...,http://scielo.isciii.es/scielo.php?script=sci_...


In [25]:
dev_rel = dev_rel.reset_index()

In [26]:
dev_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [27]:
dev_rel = dev_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [28]:
dev_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1888-75462015000400006-1,SHORT_FORM,1436,AP,SHORT-LONG,LONG_FORM,1419,1434,anteroposterior
1,S0210-56912009000700006-2,SHORT_FORM,876,angio-RMN,SHORT-LONG,LONG_FORM,858,874,angiorresonancia
2,S0210-56912009000700006-2,SHORT_FORM,819,RMN,SHORT-LONG,LONG_FORM,789,817,resonancia magnética nuclear
3,S1698-44472005000300015-1,SHORT_FORM,209,ATM,SHORT-LONG,LONG_FORM,177,207,articulación témporomandibular
4,S1698-44472005000300015-1,SHORT_FORM,1125,TC,SHORT-LONG,LONG_FORM,1099,1123,tomografía computerizada


In [29]:
dev_abbr = dev_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [30]:
dev_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S1130-14732005000300004-1,1216,1218,C3,tercera vértebra cervical,tercero vértebra cervical
1,S1130-14732005000300004-1,717,719,C2,segunda vértebra cervical,segundo vértebra cervical
2,S1130-14732005000300004-1,3191,3193,C3,tercera vértebra cervical,tercero vértebra cervical
3,S1130-14732005000300004-1,2867,2869,C3,tercera vértebra cervical,tercero vértebra cervical
4,S1130-14732005000300004-1,2862,2864,C2,segunda vértebra cervical,segundo vértebra cervical


In [31]:
dev_raw = read_texts("../datasets/development_set/development_set.raw_text/txt/")

In [32]:
dev_raw.head()

,nombre,texto
0,S0004-06142006000100014-1,Presentamos un paciente de 66 años de edad con...
1,S0004-06142006000500002-2,Paciente de 46 años que consultó por dolor a n...
2,S0004-06142006000500011-1,Enfermo varón de 59 años con antecedentes de e...
3,S0004-06142006000700011-1,Presentamos el caso de un varón de 61 años dia...
4,S0004-06142006000900008-1,Mujer de 45 años de edad que acude a la consul...


### Sample test

15 clinical cases

In [33]:
sample_abbr = pd.read_csv("../datasets/sample_set/clinical_cases.abbreviations.sample_set.tsv", sep = '\t')
sample_met = pd.read_csv("../datasets/sample_set/clinical_cases.metadata.sample_set.tsv", sep = '\t')
sample_rel = pd.read_csv("../datasets/sample_set/clinical_cases.relations.sample_set.tsv", sep = '\t')

In [34]:
sample_met = sample_met.rename(columns = {'# Document_ID': 'doc_id'})

In [35]:
sample_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S0004-06142006000900015-1,1,0004-0614,2006-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
1,S0004-06142006000600015-1,1,0004-0614,2006-08-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0004-06142007000700014-1,1,0004-0614,2007-09-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0004-06142007000900013-1,1,0004-0614,2007-11-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0004-06142006000200014-1,1,0004-0614,2006-03-01,Archivos Españoles de Urología (Ed. impresa) ...,http://scielo.isciii.es/scielo.php?script=sci_...


In [36]:
sample_rel = sample_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [37]:
sample_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A_EndOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S0004-06142006000700014-1,SHORT_FORM,926,929,CEA,SHORT-LONG,LONG_FORM,896,924,Antígeno Carcino Embrionario
1,S0004-06142005001000011-1,SHORT_FORM,1626,1629,ROT,SHORT-LONG,LONG_FORM,1600,1624,reflejos osteotendinosos
2,S0004-06142005001000011-1,SHORT_FORM,1715,1718,RMN,SHORT-LONG,LONG_FORM,1685,1713,resonancia magnética nuclear
3,S0004-06142005001000011-1,SHORT_FORM,1663,1666,RCP,SHORT-LONG,LONG_FORM,1639,1661,reflejo cutaneoplantar
4,S0004-06142005001000011-1,SHORT_FORM,1808,1811,LCR,SHORT-LONG,LONG_FORM,1783,1806,líquido cefalorraquídeo


In [38]:
sample_abbr = sample_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [39]:
sample_abbr.head()

,doc_id,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0004-06142005001000011-1,1034,1036,Kg,kilogramo,kilogramo
1,S0004-06142005001000011-1,1031,1033,mg,miligramo,miligramo
2,S0004-06142005001000011-1,196,199,IgA,inmunoglobulina a,inmunoglobulina a
3,S0004-06142005001000011-1,2057,2060,LCR,líquido cefalorraquídeo,líquido cefalorraquídeo
4,S0004-06142005001000011-1,1594,1598,EEII,extremidades inferiores,extremidad inferior


In [42]:
sample_raw = read_texts("../datasets/sample_set/sample_set.raw_text/")

In [43]:
sample_raw.head()

,nombre,texto
0,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
1,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...
2,S0004-06142005001000016-1,Varón de 65 años que consultaba por un síndrom...
3,S0004-06142006000200014-1,"Varón de 36 años, sin antecedentes personales ..."
4,S0004-06142006000300015-1,Paciente de 50 años con antecedente de litiasi...


### Trainning

318 clinical cases

In [44]:
train_abbr = pd.read_csv("../datasets/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')
train_met = pd.read_csv("../datasets/trainning_set/clinical_cases.metadata.training_set.tsv", sep = '\t')
train_rel = pd.read_csv("../datasets/trainning_set/clinical_cases.relations.training_set.tsv", sep = '\t')

In [45]:
train_met = train_met.rename(columns = {'# Document_ID': 'doc_id'})

In [46]:
train_met.head()

,doc_id,Case_ID,ISSN,Date,Source,Full_Text_Link
0,S1139-76322015000500009-1.txt,1.txt,1139-7632,2015-12-01,Pediatría Atención Primaria v.17 n.68 2015,http://scielo.isciii.es/scielo.php?script=sci_...
1,S1130-05582008000400007-2.txt,2.txt,1130-0558,2008-08-01,Revista Española de Cirugía Oral y Maxilofacia...,http://scielo.isciii.es/scielo.php?script=sci_...
2,S0210-48062006000100012-1.txt,1.txt,0210-4806,2006-01-01,Actas Urológicas Españolas v.30 n.1 2006,http://scielo.isciii.es/scielo.php?script=sci_...
3,S0213-12852003000500002-1.txt,1.txt,0213-1285,2003-10-01,Avances en Odontoestomatología v.19 n.5 2003,http://scielo.isciii.es/scielo.php?script=sci_...
4,S0212-71992005000400007-1.txt,1.txt,0212-7199,2005-04-01,Anales de Medicina Interna v.22 n.4 2005,http://scielo.isciii.es/scielo.php?script=sci_...


In [47]:
train_rel = train_rel.reset_index()

In [48]:
train_rel.columns = ['# Document_ID', 'Mention_A_type', 'Mention_A_StartOffset',
      'Mention_A', 'Relation_type', 'Mention_B_type',
       'Mention_B_StartOffset', 'Mention_B_EndOffset', 'Mention_B']

In [49]:
train_rel = train_rel.rename(columns = {'# Document_ID': 'doc_id'})

In [50]:
train_rel.head()

,doc_id,Mention_A_type,Mention_A_StartOffset,Mention_A,Relation_type,Mention_B_type,Mention_B_StartOffset,Mention_B_EndOffset,Mention_B
0,S1130-01082009000400014-1,SHORT_FORM,476,NPT,SHORT-LONG,LONG_FORM,454.0,474.0,nutrición parenteral
1,S1130-63432016000100009-1,SHORT_FORM,614,NIHSS,SHORT-LONG,LONG_FORM,621.0,662.0,National Institute of Health Stroke Scale
2,S1139-76322017000200007-1,SHORT_FORM,1145,CMV,SHORT-LONG,LONG_FORM,1128.0,1143.0,citomegalovirus
3,S1139-76322017000200007-1,SHORT_FORM,1243,VSG,SHORT-LONG,LONG_FORM,1206.0,1241.0,velocidad de sedimentación globular
4,S1139-76322017000200007-1,SHORT_FORM,1300,IGRA,SHORT-LONG,LONG_FORM,1267.0,1298.0,interferon-gamma release assays


In [51]:
train_abbr = train_abbr.rename(columns = {'# Document_ID': 'doc_id'})

In [52]:
train_raw = read_texts("../datasets/trainning_set/training_set.raw_text/txt/")

In [53]:
train_raw = train_raw.rename(columns = {'nombre': 'doc_id'})

In [54]:
train_raw.head()

,doc_id,texto
0,S0004-06142005000900013-1,Se trata de una mujer de 29 años sometida a un...
1,S0004-06142005000900015-1,"Varón de 36 años, sin antecedentes de interés,..."
2,S0004-06142005000900016-1,Mujer de 29 años con antecedentes de ulcus duo...
3,S0004-06142005001000011-1,Varón de 58 años de edad en el momento del tra...
4,S0004-06142005001000011-3,Mujer de 42 años en el momento de someterse a ...


In [55]:
train_raw.shape

(318, 2)

## BARR1

In [84]:
barr1_back1 = pd.read_csv("../datasets/barr1/BARR_ibereval_background1_set/BARR_ibereval_background1_set.tsv", sep = '\t', encoding = 'utf-8', error_bad_lines=False,header= None)
barr1_test = pd.read_csv("../datasets/barr1/BARR_ibereval_test_set/BARR_ibereval_test_docs.tsv", sep = '\t', encoding = 'utf-8', error_bad_lines=False)
barr1_sample = pd.read_csv("../datasets/barr1/BARR_ibereval_sample_set/BARR_ibereval_sample_docs.tsv", sep = '\t', encoding = 'utf-8', error_bad_lines=False)

b'Skipping line 111155: expected 4 fields, saw 5\nSkipping line 112063: expected 4 fields, saw 5\n'


In [75]:
barr1_back1 = barr1_back1.rename(columns = {0: 'ID', 1:'LANGUAGE', 2:'TITLE', 3:'ABSTRACT'})

In [76]:
barr1_back1.head()

,ID,LANGUAGE,TITLE,ABSTRACT
0,1,es,Toxoplasma gondii es un protozoo intracelular ...,Revisión de casos con diagnóstico de toxoplasm...
1,2,es,Actualmente los pacientes con cáncer de pulmón...,Recomendaciones para la determinación de bioma...
2,3,es,La asociación de tumor carcinoide y carcinoma ...,Adenocarcinoma y tumor carcinoide pulmonares s...
3,4,es,La linfadenitis cervical aislada por leishmani...,Linfadenitis cervical aislada por leishmaniasi...
4,5,es,Introducción. La infección intraabdominal gran...,Granulomatosis intraabdominal por Cladosporium...


In [77]:
barr1_test.head()

,ID,LANGUAGE,TITLE,ABSTRACT
0,3,es,Adenocarcinoma y tumor carcinoide pulmonares s...,La asociación de tumor carcinoide y carcinoma ...
1,4,es,Linfadenitis cervical aislada por leishmaniasi...,La linfadenitis cervical aislada por leishmani...
2,12,es,La Autoridad Europea de Seguridad Alimentaria ...,La Autoridad Europea de Seguridad Alimentaria ...
3,15,es,Reflujo gastroesofágico neonatal: presentación...,El hipo se define como contracciones espasmódi...
4,18,es,Normalización y validación de la batería neuro...,"Introducción. Las baterías actuales, como la B..."


In [85]:
barr1_sample.head()

,ID,LANGUAGE,ABSTRACT,TITLE,Unnamed: 4
0,1741,es,Las secuelas en la articulación temporomandibu...,Propuesta de una metodología de exploración y ...,NaN
1,3816,es,Introducción: La amputación interileoabdominal...,"Hemipelvectomía, protetización y valoración de...",NaN
2,4600,es,Pacientes y métodos: Se analizan de forma retr...,Discectomía cervical y artrodesis intersomátic...,NaN
3,5732,en,Background: The current limited evidence may b...,Randomized controlled study of a mandibular ad...,NaN
4,10086,es,Este artículo presenta la validación del ?Cues...,Emotional intelligence in the portuguese acade...,NaN
